In [ ]:
import os
import re
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 저장 경로지정
save_directory = "pdf_files"
os.makedirs(save_directory, exist_ok=True)

# 크롬 드라이버 설정
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": os.path.abspath(save_directory),
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True
})
options.add_argument("--headless=new")

# ✅ Service 객체 명시
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# JSON 파일에서 로드함
with open("patent_cases_all_pages.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 각 게시글 페이지에서 다운로드 수행
for item in data:
    title = item["제목"].strip()
    url = item["링크"]
    safe_title = re.sub(r'[\\/*?:"<>|]', "", title)
    print(f"📄 처리 중: {title}")

    try:
        driver.get(url)

        # 다운로드 링크 로딩 대기
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//a[starts-with(@href, 'javascript:download')]"))
        )
        link_elem = driver.find_element(By.XPATH, "//a[starts-with(@href, 'javascript:download')]")

        # 다운로드 트리거
        link_elem.click()
        print(f"✅ 다운로드 시작됨: {safe_title}")

        # 다운로드 대기
        time.sleep(5)

    except Exception as e:
        print(f"❌ 다운로드 실패: {title} - {e}")

# 브라우저 종료
driver.quit()
print("\n📁 전체 PDF 다운로드 완료")
